# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846701647412                   -0.70    5.0
  2   -7.852280811919       -2.25       -1.53    1.0
  3   -7.852645911837       -3.44       -2.53    3.2
  4   -7.852646673126       -6.12       -3.35    2.5
  5   -7.852646685526       -7.91       -4.68    1.2
  6   -7.852646686728       -8.92       -5.25    4.0
  7   -7.852646686729      -11.77       -5.91    1.0
  8   -7.852646686730      -12.41       -7.34    2.0
  9   -7.852646686730   +    -Inf       -7.71    3.0
 10   -7.852646686730   +    -Inf       -8.72    2.0
 11   -7.852646686730   +  -15.05       -9.63    2.2
 12   -7.852646686730   +    -Inf       -8.58    1.0
 13   -7.852646686730      -15.05       -8.61    2.0
 14   -7.852646686730   +    -Inf       -8.50    2.2
 15   -7.852646686730      -14.75       -9.37    1.2
 16   -7.852646686730   +  -14.75       -9.13    1.0
 17   -7.852646686730   +    -Inf       -9.76 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846852213167                   -0.70           4.8
  2   -7.852528614244       -2.25       -1.63   0.80    2.0
  3   -7.852637739303       -3.96       -2.70   0.80    1.0
  4   -7.852646292364       -5.07       -3.32   0.80    2.0
  5   -7.852646681872       -6.41       -4.17   0.80    1.5
  6   -7.852646686448       -8.34       -4.84   0.80    1.5
  7   -7.852646686720       -9.57       -5.81   0.80    1.8
  8   -7.852646686730      -11.02       -6.47   0.80    2.2
  9   -7.852646686730      -13.00       -7.26   0.80    1.2
 10   -7.852646686730      -13.88       -7.90   0.80    2.0
 11   -7.852646686730   +  -15.05       -8.78   0.80    1.5
 12   -7.852646686730   +    -Inf       -8.79   0.80    2.2
 13   -7.852646686730   +    -Inf       -9.79   0.80    1.0
 14   -7.852646686730   +    -Inf      -10.19   0.80    2.0
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.458007e+01     3.260492e+00
 * time: 0.39711594581604004
     1     1.106978e+00     1.874839e+00
 * time: 0.6102049350738525
     2    -1.922641e+00     1.932285e+00
 * time: 0.6343128681182861
     3    -3.825514e+00     1.793098e+00
 * time: 0.6678240299224854
     4    -5.316811e+00     1.383739e+00
 * time: 0.7017550468444824
     5    -6.777217e+00     7.324062e-01
 * time: 0.7355539798736572
     6    -6.844868e+00     9.406732e-01
 * time: 0.7592949867248535
     7    -7.390490e+00     6.286716e-01
 * time: 0.7829329967498779
     8    -7.507534e+00     5.003075e-01
 * time: 0.8067479133605957
     9    -7.643853e+00     1.488940e-01
 * time: 0.830374002456665
    10    -7.706398e+00     1.009865e-01
 * time: 0.8540959358215332
    11    -7.760023e+00     1.241759e-01
 * time: 0.87813401222229
    12    -7.790368e+00     1.145642e-01
 * time: 0.9022960662841797
    13    -7.830553e+00     5.754543e-02
 * time: 0.926250934600

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846829703318                   -0.70    4.8
  2   -7.852309317659       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686708                   -2.53
  2   -7.852646686730      -10.67       -6.01
  3   -7.852646686730   +    -Inf      -13.02


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.761873251566663e-13
|ρ_newton - ρ_scfv| = 1.2868685092881056e-12
|ρ_newton - ρ_dm|   = 5.588289641017791e-10
